In [1]:
import mip


In [2]:

from itertools import product

# sum_((a1, a2) in A) c_(a1, a2) \sum_((b1, b2) in B \setminus \Delta) x[i1, j1] * x[b2, b2]


def bin_packing_frac(A, B, c, sizes):
    # c capacity of all the bins

    model = mip.Model()
    model.verbose = 0

    x = {(a, b): model.add_var(f"x_{a}_{b}", obj=0, var_type=mip.BINARY) for a, b in product(A, B)}
    y = {b: model.add_var(f"y_{b}", obj=1, var_type=mip.BINARY) for b in B}

    lambd = {(a, b): model.add_var(f"lambda_{a}_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for a, b in product(A, B)}
    alpha = {b: model.add_var(f"alpha_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for b in B}

    for b in B:
        model.add_constr(mip.quicksum(sizes[a] * lambd[a, b] for a in A) <= c * y[b])

    for a in A:
        model.add_constr(mip.quicksum(lambd[a, b] for b in B) == 1)

    for a, b in product(A, B):
        # non-linear constraint `model.add_constr(lambd[a, b] == alpha[b] * x[a, b])` is remplaced by:

        model.add_constr(lambd[a, b] <= x[a, b])
        model.add_constr(lambd[a, b] <= alpha[b])
        model.add_constr(lambd[a, b] + 1 >= alpha[b] + x[a, b])

    model.optimize()

    print(model.status)

    return {k for k, v in x.items() if v.x > 0}, {k for k, v in y.items() if v.x > 0}

bin_packing_frac({"a", "b", "c"}, {1, 2, 3}, 15, {"a": 32, "b": 4, "c": 4} )

OptimizationStatus.OPTIMAL


({('a', 1),
  ('a', 2),
  ('a', 3),
  ('b', 1),
  ('b', 2),
  ('b', 3),
  ('c', 1),
  ('c', 2),
  ('c', 3)},
 {1, 2, 3})

In [3]:
# A is now a weighted directed graph

def bin_packing_frac(A, B, c, sizes):
    # c capacity of all the bins
    
    model = mip.Model(solver_name=mip.GRB)
    
    x = {(a, b): model.add_var(f"x_{a}_{b}", obj=0, var_type=mip.BINARY) for a, b in product(A, B)}
    y = {b: model.add_var(f"y_{b}", obj=1, var_type=mip.BINARY) for b in B}
    
    lambd = {(a, b): model.add_var(f"lambda_{a}_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for a, b in product(A, B)}
    alpha = {b: model.add_var(f"alpha_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for b in B}
    
    for b in B:
        model.add_constr(mip.quicksum(sizes[a] * lambd[a, b] for a in A) <= c * y[b])
    
    for a in A:
        model.add_constr(mip.quicksum(lambd[a, b] for b in B) == 1)
    
    for a, b in product(A, B):
        # non-linear constraint `model.add_constr(lambd[a, b] == alpha[b] * x[a, b])` is remplaced by:

        model.add_constr(lambd[a, b] <= x[a, b])
        model.add_constr(lambd[a, b] <= alpha[b])
        model.add_constr(lambd[a, b] + 1 >= alpha[b] + x[a, b])

    model.optimize()

    print(model.status)

    return {k for k, v in x.items() if v.x > 0}, {k for k, v in y.items() if v.x > 0}

bin_packing_frac({"a": {"b": 1}, "b", "c"}, {1, 2, 3}, 15, {"a": 32, "b": 4, "c": 4} )

SyntaxError: ':' expected after dictionary key (2404558720.py, line 33)